This file aims to gain:
1. the dataframe of the diversity of each accommodation from the accommodation list
2. the dataframe of the scores of each accommodation according a scoring system
3. the dataframe of the distance between each accommodation and 4 different schools(LSE,UCL,KCL,LCF).

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import googlemaps
import regex as re
import pickle
from plotnine import *

In [3]:
df_halls= pd.read_pickle('four_brands_halls')

In [4]:
_dic = df_halls.to_dict()
location_dic = {}
for i in range(len(_dic['latitude'])):
    location_dic[i] = [_dic['latitude'][i],_dic['longitude'][i]]

In [5]:
check = pd.read_pickle('crime rate rank and time to places')

In [6]:
def get_diversity(location):
    _result = []
    final_list = []
    keywords = ['restaurant','pub','bar','club','college','river','church','education']
    url = f"https://api.foursquare.com/v3/places/search?fields=categories&ne={float(location[0])+0.002}%2C{float(location[1])+0.002}&sw={float(location[0])-0.002}%2C{float(location[1])-0.002}&sort=DISTANCE&limit=50"
    headers = {
    "accept": "application/json",
    "Authorization": "fsq3hthzjz5pJPEqfceTlq/8jDalidYiFXMfMUUpJOUo7Qk="
    }

    response = requests.get(url,headers=headers)
    _list = json.loads(response.text)
    
    for i in _list['results']:
        if len(i['categories']) == 1:
            _result.append(i['categories'][0]['name'].lower())
        elif len(i['categories'])>1:
            small_list = [w['name'] for w in i['categories']]
            for each in small_list:
                _result.append(each.lower())
    for store in _result:
        matched = False
        for word in store.split():
            if word in keywords:
                final_list.append(word)
                matched = True
                break
        if not matched:
            final_list.append(store)
    return len(final_list)

In [7]:
diversity_list = []
for location in location_dic:
    diversity_list.append(get_diversity(location_dic[location]))
diver_df = pd.DataFrame({'diversity':diversity_list})

In [8]:
diver_df.to_pickle('diversity type')

In [9]:
pure_number = check.join(diver_df)
pure_number.to_pickle('pure number df')

In [89]:
pd.read_pickle('pure number df')

,name,crime rate score,avg time to nearby convenience stores,avg time to nearby supermarkets,avg time to nearby restaurants,avg time to nearby bus stops,avg time to nearby underground station,avg time to nearby pharmacy,diversity
0,Urbanest Canal Reach,81.84925,8.40,7.03,4.17,1.10,12.08,11.80,23
1,Urbanest Hoxton,82.49350,2.55,2.72,3.87,0.02,5.58,7.32,51
2,Urbanest Westminster Bridge,97.25050,4.37,3.70,2.68,2.10,6.35,5.90,53
3,Urbanest St Pancras,79.81875,7.40,10.20,8.23,8.65,12.35,12.38,15
4,Urbanest City,97.38350,2.60,5.55,1.87,1.05,4.67,11.07,57
...,...,...,...,...,...,...,...,...,...
73,Scape Kings Cross,92.06825,4.70,4.55,1.57,4.23,7.33,5.65,59
74,Scape Shoreditch,91.88650,4.68,4.97,3.38,1.55,2.60,8.42,55
75,Scape Mile End,84.66200,10.22,13.83,12.80,13.63,10.18,15.63,16
76,Scape Wembley,97.15825,5.97,4.93,3.82,2.35,4.58,11.73,47


In [45]:
sdf = pd.read_pickle('pure number df')

In [46]:
sdf = sdf.drop(sdf.index[[10,77,32]])#drop NaN values
sdf['convenience'] = sdf[['avg time to nearby convenience stores','avg time to nearby supermarkets','avg time to nearby restaurants']].mean(axis=1)
sdf['transportation'] = sdf.apply(lambda row: min(row['avg time to nearby bus stops'], row['avg time to nearby underground station']), axis=1)
sdf = sdf.drop(columns=['avg time to nearby convenience stores','avg time to nearby supermarkets',
                 'avg time to nearby restaurants','avg time to nearby bus stops','avg time to nearby underground station'])

In [31]:
sdf.max()

name                                      urbanest Victoria
crime rate score                                   98.48275
avg time to nearby convenience stores                 10.87
avg time to nearby supermarkets                       13.83
avg time to nearby restaurants                        13.67
avg time to nearby bus stops                          13.63
avg time to nearby underground station                75.68
avg time to nearby pharmacy                            27.3
diversity                                                66
convenience                                       12.283333
transport                                             10.18
dtype: object

In [47]:
def calculate_five_point_score(column):
    try:
        numeric_column = pd.to_numeric(column)
    except ValueError:
        return column
    if 'crime rate score' in column.name or 'diversity' in column.name:
        score_range = (numeric_column.max()-numeric_column.min())/5
        return ((column - column.min()) / score_range).round(3)
    else:
        score_range = (numeric_column.max() - numeric_column.min()) / 5
        return ((numeric_column.max() - numeric_column) / score_range).round(3)

In [48]:
scored_df = sdf.apply(lambda column: calculate_five_point_score(column))

In [49]:
scored_df['brand'] = sdf['name'].apply(lambda i: 'Unite Students' if 'Unite Students' in i 
                                 else 'iQ' if 'iQ' in i 
                                 else 'Scape' if 'Scape' in i 
                                 else 'Urbanest' if 'Urbanest' in i 
                                 else 'Chapter' if 'Chapter' in i
                                 else None)
scored_df.rename(columns={'avg time to nearby pharmacy':'convenience(pharmacy)','convenience':'convenience(food & dairy products)'},inplace=True)

In [50]:
scored_df

,name,crime rate score,convenience(pharmacy),diversity,convenience(food & dairy products),transportation,brand
0,Urbanest Canal Reach,1.791,3.471,1.019,2.880,4.469,Urbanest
1,Urbanest Hoxton,1.915,4.474,3.611,4.626,5.000,Urbanest
2,Urbanest Westminster Bridge,4.762,4.792,3.796,4.357,3.976,Urbanest
3,Urbanest St Pancras,1.399,3.341,0.278,1.840,0.753,Urbanest
4,Urbanest City,4.788,3.634,4.167,4.479,4.493,Urbanest
...,...,...,...,...,...,...,...
72,Scape Bloomsbury,3.567,4.709,4.074,3.699,4.385,Scape
73,Scape Kings Cross,3.762,4.848,4.352,4.346,2.928,Scape
74,Scape Shoreditch,3.727,4.227,3.981,3.977,4.247,Scape
75,Scape Mile End,2.333,2.613,0.370,0.000,0.000,Scape


In [52]:
scored_df['total_score'] = scored_df.apply(lambda x: 
    round(float(x['crime rate score'])* 0.242 +    
    float(x['convenience(pharmacy)'])* 0.132+
    float(x['diversity'])* 0.128+
    float(x['convenience(food & dairy products)'])* 0.231+
    float(x['transportation'])* 0.267,3), axis=1)

In [54]:
scored_df.to_pickle('scored_df')

In [70]:
pd.read_pickle('distance to schools')

,name,LSE,KCL,UCL,LCF
0,Urbanest Canal Reach,4.0,4.9,2.7,4.2
1,Urbanest Hoxton,3.6,4.3,4.8,6.3
2,Urbanest Westminster Bridge,2.4,2.0,4.5,4.3
3,Urbanest St Pancras,3.5,4.3,2.2,3.6
4,Urbanest City,3.6,3.2,6.0,7.5
...,...,...,...,...,...
73,Scape Kings Cross,2.0,2.9,1.5,3.0
74,Scape Shoreditch,3.4,4.0,4.6,6.1
75,Scape Mile End,8.3,8.0,8.7,10.7
76,Scape Wembley,20.8,21.7,18.7,18.7
